#### PPGIA - Unifor - Prof. Daniel - Processamento e analise de imagens digitais - TrabalhoFinal01 - Separação e Criação de Atas

#### Importação de Bibliotecas

In [ ]:
import numpy as np
import pandas as pd
import cv2 
print(cv2.__version__)
#Path Google Colab para CV2
from google.colab.patches import cv2_imshow
from google.colab import files
import dlib
from skimage import io
from PIL import Image 
%matplotlib inline
#A linha acima é necessária para mostrar gráficos Matplotlib dentro de um Notebook Jupyter
from matplotlib import pyplot as plt
#import matplotlib.pylab as plt
import sys
import ipywidgets as widgets
from IPython.display import display
import scipy
from scipy import interpolate
#import face_recognition
import pickle
import gzip

4.1.2


In [ ]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelBinarizer
from keras.models import Sequential
from keras.layers.core import Dense
from tensorflow.keras.optimizers import SGD
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras

#### Upload do arquivo

In [ ]:
#Upload do arquivo de imagens que será utilizado no exemplo (lena.png)
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving Atlas.png to Atlas.png
User uploaded file "Atlas.png" with length 27690 bytes


#### Funções

In [ ]:
#Funcao para Exibir Imagens com matplotlib e cv2
def exibir_imgplt(tmpimg):
  #Importar a Imagem
  image = cv2.imread(tmpimg, cv2.IMREAD_UNCHANGED)
  #Aplicando a máscara de onde os bits transparentes estão
  trans_mask = image[:,:,3] == 0
  #substitua as áreas de transparência por branco e não transparente
  image[trans_mask] = [255, 255, 255, 255]
  image = cv2.cvtColor(image, cv2.COLOR_BGRA2BGR)
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  #Mostra a imagem com matplotlib
  plt.figure(figsize = (100,10))
  plt.imshow(image)
  plt.show()
  return


#Funcao para Exibir Imagens com matplotlib e cv2
def exibir_atlas(tmpimg):
  #Importar a Imagem
  image = cv2.imread(tmpimg, cv2.IMREAD_UNCHANGED)
  #Aplicando a máscara de onde os bits transparentes estão
  trans_mask = image[:,:,3] == 0
  #substitua as áreas de transparência por branco e não transparente
  image[trans_mask] = [255, 255, 255, 255]
  image = cv2.cvtColor(image, cv2.COLOR_BGRA2BGR)
  imagegray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

  #Aplicando a detecção canny edge 
  edged = cv2.Canny(imagegray, 10, 250)

  contours, hierarchy = cv2.findContours(edged.copy(),cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
  #rect = cv2.minAreaRect(contours[0])

  x, y = [], []

  for contour_line in contours:
      for contour in contour_line:
          x.append(contour[0][0])
          y.append(contour[0][1])

  x1, x2, y1, y2 = min(x), max(x), min(y), max(y)

  cropped = image[y1:y2, x1:x2]  

  #countours_res = []
  #for i in range(0, len(contours)):
  #  area = cv2.contourArea(contours[i])
  #  rect = cv2.minAreaRect(contours[i])
  #  box = cv2.boxPoints(rect)
  #  cv2.drawContours(image,[box],0,(0,0,255),2)
    #x, y, w, h = cv2.boundingRect(contours[i])
    #x, y, w, h = point_judge([int(rect[0][0]), int(rect[0][1])], box)
    #src = np.float32([x, y, w, h])
    #cv2.imwrite('img_{}.jpg'.format(i), image[y:y+h,x:x+w])

  #Mostra a imagem com matplotlib
  plt.figure(figsize = (100,10))
  plt.imshow(cropped)
  plt.show()

  while True:
      if cv2.waitKey(6) & 0xff == 27:
          break 

  return 

In [ ]:
#Botão para exibir a imagem que foi realizado Upload
button = widgets.Button(description="Exibir Img CV2PLT")
output = widgets.Output()

def on_button_clicked_exibir_imagem(b):
  with output:
    img = cv2.cvtColor(cv2.imread(next(iter(uploaded))), cv2.COLOR_BGR2RGB)
    print("Exibindo Imagens:")
    print("Resolução: ", img.shape[0: 2])
    #print(img) # Imprimindo a imagem como NumpyArray
    exibir_imgplt(next(iter(uploaded)))
button.on_click(on_button_clicked_exibir_imagem)
display(button, output)

Button(description='Exibir Img CV2PLT', style=ButtonStyle())

Output()

In [ ]:
#Botão para chamar a função para detectar, cortar e exibir as imagens do ATLAS
button2 = widgets.Button(description="Exibir Atlas")
output2 = widgets.Output()

def on_button_clicked_exibir_atlas(b):
  with output2:
    exibir_atlas(next(iter(uploaded)))
button2.on_click(on_button_clicked_exibir_atlas)
display(button2, output2)

Button(description='Exibir Atlas', style=ButtonStyle())

Output()